In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Handwriting-Transformers

/content/drive/MyDrive/Handwriting-Transformers


In [ ]:
# !git clone https://github.com/ankanbhunia/Handwriting-Transformers
# %cd Handwriting-Transformers
# !pip install --upgrade --no-cache-dir gdown
# !gdown --id 16g9zgysQnWk7-353_tMig92KsZsrcM6k && unzip files.zip && rm files.zip

Cloning into 'Handwriting-Transformers'...
remote: Enumerating objects: 497, done.
remote: Counting objects: 100% (112/112), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 497 (delta 54), reused 72 (delta 41), pack-reused 385
Receiving objects: 100% (497/497), 67.85 MiB | 19.33 MiB/s, done.
Resolving deltas: 100% (197/197), done.
/content/drive/MyDrive/Handwriting-Transformers
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0
/usr/local/lib/python3.8/dist-packages/gdown/cli.py:121: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=16g9zgysQnWk7-353_tMig92KsZsrcM6k
To: /content/drive/MyDrive/Handwriting-Transformer

In [ ]:
import os
import time
from data.dataset import TextDataset, TextDatasetval
import torch
import cv2
import os
import numpy as np
from models.model import TRGAN
from params import *
from torch import nn
from data.dataset import get_transform
import pickle
from PIL import Image
import tqdm
import shutil

In [ ]:
text = "Whenever you feel like criticizing anyone, just remember that all the people in this world haven't had the advantages that you've had"
output_path = 'results'

In [ ]:
model_path = 'files/iam_model.pth'; data_path = 'files/IAM-32.pickle' #(iam)
# model_path = 'files/cvl_model.pth'; data_path = 'files/CVL-32.pickle' #(cvl)
# model_path = 'files/iam_model.pth'; data_path = 'files/CVL-32.pickle' #(iam-cvl-cross)
#model_path = 'files/cvl_model.pth'; data_path = 'files/IAM-32.pickle' #(cvl-iam-cross)#

In [ ]:
print ('(1) Loading dataset files...')

TextDatasetObjval = TextDatasetval(base_path = data_path, num_examples = 15)
datasetval = torch.utils.data.DataLoader(
            TextDatasetObjval,
            batch_size=batch_size,
            shuffle=True,
            num_workers=0,
            pin_memory=True, drop_last=True,
            collate_fn=TextDatasetObjval.collate_fn)

print ('(2) Loading model...')

model = TRGAN()
model.netG.load_state_dict(torch.load(model_path))
print (model_path+' : Model loaded Successfully')

print ('(3) Loading text content...')
text_encode =  [j.encode() for j in text.split(' ')]
eval_text_encode, eval_len_text = model.netconverter.encode(text_encode)
eval_text_encode = eval_text_encode.to('cuda:0').repeat(batch_size, 1, 1)


(1) Loading dataset files...
(2) Loading model...


/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


initialize network with N02
initialize network with N02
initialize network with N02


/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/usr/local/lib/python3.8/dist-packages/torchvision/models/inception.py:43: FutureWarning: The default weight initialization of inception_v3 will be changed in future releases of torchvision. If you wish to keep the old behavior (which leads to long initialization times due to scipy/scipy#11299), please set init_weights=True.
  warnings.warn(


files/iam_model.pth : Model loaded Successfully
(3) Loading text content...


In [ ]:
batch_size

8

In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

%matplotlib inline

In [ ]:
import os
file_list = os.listdir('/content/drive/MyDrive/Handwriting-Transformers/files/exp2/')

In [ ]:
file_list

['2.png',
 '7.png',
 'KakaoTalk_20221230_141737354_01.png',
 'KakaoTalk_20221230_141737354.png',
 'KakaoTalk_20221230_142116248_01.jpg',
 'KakaoTalk_20221230_142116248.jpg',
 'KakaoTalk_20221230_142340734.png',
 'KakaoTalk_20221230_142531134.jpg']

In [ ]:
imgs_pad = []
imgs_wids = []
for i in range(len(file_list)):
  img = Image.open("/content/drive/MyDrive/Handwriting-Transformers/files/exp2/" + str(file_list[i]))
  img = np.array(img)
  img = img[:, :, 0]
  scaler = StandardScaler()
  img = scaler.fit_transform(img)
  img = cv2.resize(img, (768, 32))
  img = torch.Tensor(img).to(DEVICE)
  temp = torch.stack((img, img, img, img, img, img, img, img, img, img, img, img, img, img, img), 0)
  imgs_pad.append(temp)
  imgs_wids.append(temp[:, 0, 0])

In [ ]:
img = torch.stack((imgs_pad[0], imgs_pad[1], imgs_pad[2], imgs_pad[3], imgs_pad[4], imgs_pad[5], imgs_pad[6], imgs_pad[7]), 0)

In [ ]:
img2 = torch.stack((imgs_wids[0], imgs_wids[1], imgs_wids[2], imgs_wids[3], imgs_wids[4], imgs_wids[5], imgs_wids[6], imgs_wids[7]), 0)

In [ ]:
print(batch_size, img.shape, img2.shape)

8 torch.Size([8, 15, 32, 768]) torch.Size([8, 15])


In [ ]:
page_val = model._generate_page(img, img2, eval_text_encode,eval_len_text)
cv2.imwrite(output_path+'/image' + '20221230-exp5' + '.png', page_val*255)

True

In [ ]:
imgs_wids[0].shape

torch.Size([15])

In [ ]:
imgs_pad[0].shape

torch.Size([15, 32, 768])